In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import svm

from scipy.stats import loguniform
from pyspark.ml.feature import VectorAssembler

from pyspark.sql.functions import col


from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator


import sys
sys.path.append('../../DataPreprocessing')
sys.path.append('../Classification')
import DataPreprocessing
from utils import read_data, hyperparameter_search, encode, decode, floor_col

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[1]") \
    .appName("aymon") \
    .getOrCreate()


In [3]:
df = DataPreprocessing.read_data(spark,file_name='../../Google-Playstore.csv', absolute_csv_path=True)
df= DataPreprocessing.convert_size_to_bytes(df)
df = DataPreprocessing.remove_useless_col(df, ['Scraped time', 'App Name', 'App Id', 'Minimum Installs', 'Maximum Installs', 'Currency', 'Developer Email'\
                                               , 'Developer Id'])
#developer Id might be beneficial, but it is a nominal categorized feature, we cannot grasp its potential, rather we can make another feature
#as number of apps developed for each developer


In [4]:
#DataPreprocessing.show_nulls(df)

In [5]:

uninteresting_cols= ['Minimum Android','Size','Installs','Price','Ad Supported','In App Purchases', 'Released', 'Last Updated']
df=DataPreprocessing.handle_missing_values(df,cols=uninteresting_cols)
interesting_num_cols=['Rating','Rating Count', 'Installs']
df= DataPreprocessing.handle_missing_values(df, handling_method='mean', cols=interesting_num_cols)
df = DataPreprocessing.binarize_col(df, cols=['Privacy Policy', 'Developer Website'])
df = DataPreprocessing.convert_binary_pyspark(df, cols=['Ad Supported', 'In App Purchases', 'Editors Choice', 'Free'])
df = df.filter("Rating>=1")
df = df.filter(col('Rating Count') >=200)
df.show(10)

Total Number of rows : 2237972
Number of rows after dropping nulls: 2183417
Total Number of rows : 2183417
+-----------------+------+------------+--------+-----+-----+---------------+------------+------------+--------------+--------------+-----------------+------------+----------------+--------------+----+
|         Category|Rating|Rating Count|Installs|Price| Size|Minimum Android|    Released|Last Updated|Content Rating|Privacy Policy|Developer Website|Ad Supported|In App Purchases|Editors Choice|Free|
+-----------------+------+------------+--------+-----+-----+---------------+------------+------------+--------------+--------------+-----------------+------------+----------------+--------------+----+
|  Personalization|   4.7|         820|   50000|  0.0|  3.5|     4.1 and up|Sep 22, 2019|Oct 07, 2020|      Everyone|          true|             true|        true|           false|         false|true|
|   Travel & Local|   3.7|        1572|   10000|  0.0|2.9E7|     4.2 and up| Sep 5, 2018|

In [6]:
df.printSchema()

root
 |-- Category: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Rating Count: integer (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Price: float (nullable = true)
 |-- Size: float (nullable = true)
 |-- Minimum Android: string (nullable = true)
 |-- Released: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Privacy Policy: boolean (nullable = false)
 |-- Developer Website: boolean (nullable = false)
 |-- Ad Supported: boolean (nullable = false)
 |-- In App Purchases: boolean (nullable = false)
 |-- Editors Choice: boolean (nullable = false)
 |-- Free: boolean (nullable = false)



In [7]:
df = DataPreprocessing.convert_Last_Updated_to_Year(df)
df.show(10)

Total Number of rows : 267147
Number of rows after dropping nulls: 267141
+------------+------+
|Last Updated| count|
+------------+------+
|        2021|107641|
|        2020| 70608|
|        2019| 32809|
|        2018| 20270|
|        2017| 13441|
|        2016|  8978|
|        2015|  6490|
|        2014|  4109|
|        2013|  1913|
|        2012|   586|
|        2011|   221|
|        2010|    73|
|        2009|     2|
+------------+------+

+-----------------+------+------------+--------+-----+-----+---------------+------------+------------+--------------+--------------+-----------------+------------+----------------+--------------+----+
|         Category|Rating|Rating Count|Installs|Price| Size|Minimum Android|    Released|Last Updated|Content Rating|Privacy Policy|Developer Website|Ad Supported|In App Purchases|Editors Choice|Free|
+-----------------+------+------------+--------+-----+-----+---------------+------------+------------+--------------+--------------+-----------------

In [8]:
#DataPreprocessing.get_info(df)

In [9]:
df = encode(df, columns=['Category', 'Minimum Android', 'Content Rating'])
#df = floor_col(df, 'Rating')
df.show(10)

+-----------------+------+------------+--------+-----+-----+---------------+------------+------------+--------------+--------------+-----------------+------------+----------------+--------------+----+------------+-------------------+------------------+
|         Category|Rating|Rating Count|Installs|Price| Size|Minimum Android|    Released|Last Updated|Content Rating|Privacy Policy|Developer Website|Ad Supported|In App Purchases|Editors Choice|Free|Category_enc|Minimum Android_enc|Content Rating_enc|
+-----------------+------+------------+--------+-----+-----+---------------+------------+------------+--------------+--------------+-----------------+------------+----------------+--------------+----+------------+-------------------+------------------+
|  Personalization|   4.7|         820|   50000|  0.0|  3.5|     4.1 and up|Sep 22, 2019|        2020|      Everyone|          true|             true|        true|           false|         false|true|           3|                  0|        

In [10]:
df.printSchema()

root
 |-- Category: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Rating Count: integer (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Price: float (nullable = true)
 |-- Size: float (nullable = true)
 |-- Minimum Android: string (nullable = true)
 |-- Released: string (nullable = true)
 |-- Last Updated: integer (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Privacy Policy: boolean (nullable = false)
 |-- Developer Website: boolean (nullable = false)
 |-- Ad Supported: boolean (nullable = false)
 |-- In App Purchases: boolean (nullable = false)
 |-- Editors Choice: boolean (nullable = false)
 |-- Free: boolean (nullable = false)
 |-- Category_enc: integer (nullable = true)
 |-- Minimum Android_enc: integer (nullable = true)
 |-- Content Rating_enc: integer (nullable = true)



In [11]:
inputColums=['Rating Count','Installs','Price',\
             'Size','Last Updated','Privacy Policy','Developer Website',\
                'Ad Supported','In App Purchases','Editors Choice','Free','Category_enc','Minimum Android_enc','Content Rating_enc']
assembler = VectorAssembler(inputCols=inputColums, outputCol='features')

traindf = assembler.transform(df).select(['features', 'Rating'])
traindf.show(10)

+--------------------+------+
|            features|Rating|
+--------------------+------+
|[820.0,50000.0,0....|   4.7|
|[1572.0,10000.0,0...|   3.7|
|(14,[0,1,3,4,5,10...|   3.7|
|[211.0,10000.0,0....|   4.4|
|[736.0,500000.0,0...|   3.8|
|[420.0,50000.0,0....|   2.3|
|(14,[0,1,3,4,5,6,...|   2.7|
|[2781.0,100000.0,...|   4.4|
|[266.0,10000.0,0....|   4.3|
|[337.0,100000.0,0...|   4.1|
+--------------------+------+
only showing top 10 rows



In [12]:

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = traindf.randomSplit([0.7, 0.3])

# Train a RandomForest model.
rf = RandomForestRegressor(featuresCol="features", labelCol='Rating')

# Train model.  This also runs the indexer.
model = rf.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "Rating", "features").show(5)

predictionAndTarget = predictions.select(['Rating', 'prediction'])


+------------------+------+--------------------+
|        prediction|Rating|            features|
+------------------+------+--------------------+
| 4.297017480882392|   4.7|(14,[0,1,2,3,4],[...|
|3.9688648529891575|   4.2|(14,[0,1,2,3,4,5]...|
|4.1952177321708986|   4.5|(14,[0,1,2,3,4,5]...|
| 4.268224839614224|   4.7|(14,[0,1,2,3,4,5,...|
| 4.161377089041426|   4.7|(14,[0,1,2,3,4,5,...|
+------------------+------+--------------------+
only showing top 5 rows



In [13]:

evaluator = RegressionEvaluator(
    labelCol="Rating", predictionCol="prediction", metricName="r2")
rmse = evaluator.evaluate(predictionAndTarget)
print("r2 error (r2) on test data = %g" % rmse)


evaluator = RegressionEvaluator(
    labelCol="Rating", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictionAndTarget)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

r2 error (r2) on test data = 0.111925
Root Mean Squared Error (RMSE) on test data = 0.55404
